# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [95]:
# calculate the cosine distance between all pairs of time series

import numpy as np
from scipy import spatial, linalg, stats

n_timepoints = 652
n_regions = 6
timeseries = np.random.rand(n_regions, n_timepoints)

dist = spatial.distance.pdist(timeseries, 'correlation')

# np.isclose(1 - dist, pearson_corr[np.triu_indices(n_regions, k=1)])

pearson_corr = np.corrcoef(timeseries)
spearman_corr = stats.spearmanr(timeseries, axis=1).correlation

corr_inv = linalg.inv(spearman_corr)

pcorr = np.zeros_like(spearman_corr)
for i in range(corr_inv.shape[0]):
    for j in range(corr_inv.shape[1]):
        if i != j:  # off-diagonal
            pcorr[i, j] = -corr_inv[i, j] / np.sqrt(corr_inv[i, i] * corr_inv[j, j])

pcorr[np.diag_indices_from(pcorr)] = 1.0  # set diagonal to 1


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
%reload_ext autoreload
%autoreload 2

import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [20]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]

# DEBUG single epoch, one train/test split
model = MultiHeadModel(n_regions, n_networks, n_embeddings=32)
trainer = pl.Trainer(accelerator='auto',
                     max_epochs=10000,
                     accumulate_grad_batches=3,
                     gradient_clip_val=.5,
                     log_every_n_steps=3,
                     callbacks=[RichProgressBar()])
trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

(160, 6)